In [13]:
import torch
# from vision.ssd.vgg_ssd import create_vgg_ssd, create_vgg_ssd_predictor
# from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd, create_mobilenetv1_ssd_predictor
# from vision.ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite, create_mobilenetv1_ssd_lite_predictor
# from vision.ssd.squeezenet_ssd_lite import create_squeezenet_ssd_lite, create_squeezenet_ssd_lite_predictor
# from vision.datasets.voc_dataset import VOCDataset
# from vision.datasets.open_images import OpenImagesDataset
# from vision.utils import box_utils, measurements
# from vision.utils.misc import str2bool, Timer
import argparse
import pathlib
import numpy as np
import logging
import sys
# from vision.ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite, create_mobilenetv2_ssd_lite_predictor

import os
import torch.quantization


In [30]:
class Args():
    net="vgg16"
    #model="./ssd-mobilenet-fruit/mb1-ssd-Epoch-99-Loss-3.454277849197388.pth"
    model = "./checkpoint/vgg16/Sunday_04_December_2022_03h_07m_25s/vgg16-200-regular.pth"
    #dataset_type="open_images"
    #dataset="data/fruit"
    checkpoint_folder="./quantized_model/"
    gpu=False

args=Args()



In [31]:
from utils import get_network
net = get_network(args)
print(f"Loading model {args.model}")
ckp=torch.load(args.model)
# logging.info(f"loading model {args.model}")
net.load_state_dict(ckp)


Loading model ./checkpoint/vgg16/Sunday_04_December_2022_03h_07m_25s/vgg16-200-regular.pth


<All keys matched successfully>

In [8]:

# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() and args.use_cuda else "cpu")
# load the dataset
if args.dataset_type == "voc":
    dataset = VOCDataset(args.dataset, is_test=True)
elif args.dataset_type == 'open_images':
    dataset = OpenImagesDataset(args.dataset, dataset_type="test")
else:
    print(f"The dataset type  {args.dataset_type } is not supported.")


# create the network
if args.net == 'vgg16-ssd':
    net = create_vgg_ssd(len(dataset.class_names), is_test=True)
elif args.net == 'mb1-ssd':
    net = create_mobilenetv1_ssd(len(dataset.class_names), is_test=True)
elif args.net == 'mb1-ssd-lite':
    net = create_mobilenetv1_ssd_lite(len(dataset.class_names), is_test=True)
elif args.net == 'sq-ssd-lite':
    net = create_squeezenet_ssd_lite(len(dataset.class_names), is_test=True)
elif args.net == 'mb2-ssd-lite':
    net = create_mobilenetv2_ssd_lite(len(dataset.class_names), width_mult=args.mb2_width_mult, is_test=True)
else:
    logging.fatal(f"Invalid network architecture type '{arch}' - it should be one of:  vgg16-ssd, mb1-ssd, mb1-ssd-lite, mb2-ssd-lite, sq-ssd-lite")
    sys.exit(1)

# load the model
print(f"Loading model {args.model}")
# logging.info(f"loading model {args.model}")
net.load(args.model)
# net = net.to(DEVICE)
# logging.info(f"loaded model {args.model}")

num images:  930
Loading model models/fruit/mb1-ssd-Epoch-68-Loss-3.3787471590635603.pth


In [32]:
print(net)
# print(net.named_modules)
# # print(net.named_parameters)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [33]:
# net.base_net.half()
net.half()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [34]:
model_path = os.path.join(args.checkpoint_folder, f"{args.net}-quant-all-half.pth")
torch.save(net.state_dict() ,model_path)

In [19]:
for i in net.base_net.modules():
    if isinstance(i, torch.nn.Conv2d):
        # print(i.weight)
        print((i.weight).dtype)
        # break

AttributeError: 'MobileNet' object has no attribute 'base_net'

In [20]:
for i in net.base_net.modules():
    if isinstance(i, torch.nn.Conv2d):
        # print(i.weight)
        print((i.weight).dtype)
        break
    

AttributeError: 'MobileNet' object has no attribute 'base_net'

In [14]:

quantized_base_net = torch.quantization.quantize_dynamic(
    net.base_net, {torch.nn.Conv2d,torch.nn.Linear}, dtype=torch.qint8
)

net.base_net = quantized_base_net

# print("Quantized model")
# run_model(quantized_model, dummy_input)

In [21]:
net.base_net.half()
# change dtype of the model to float16
for i in net.base_net.modules():
    if isinstance(i, torch.nn.Conv2d):
        # print(i.weight)

        print((i.weight).dtype)
        break

AttributeError: 'MobileNet' object has no attribute 'base_net'

In [16]:

model_path = os.path.join(args.checkpoint_folder, f"{args.net}-quant-base_net.pth")

net.save(model_path)



In [22]:
for module in [net.base_net,net.extras,net.regression_headers,net.classification_headers]:
    # for m in module.modules():
    #     if type(m) == torch.nn.Conv2d:
    #         m.qconfig = torch.quantization.default_qconfig
    #         torch.quantization.prepare(m, inplace=True)
    quantized_base_net = torch.quantization.quantize_dynamic(    module, {torch.nn.Conv2d,torch.nn.Linear}, dtype=torch.qint8)
    module = quantized_base_net

model_path = os.path.join(args.checkpoint_folder, f"{args.net}-quant-all.pth")
net.save(model_path)

AttributeError: 'MobileNet' object has no attribute 'base_net'